In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine, types
import psycopg2
import pymysql

In [3]:
indata=pd.read_csv("../dataset2/etl_param.csv")

In [4]:
indata

,PREFIX,DBNAME,IP,PORT,ID,PW,PATH,PARAM_CATEGORY,USE_YN
0,oracle+cx_oracle,orcl,192.168.110.111,1521,TAEYEON,TAEYEON,../dataset2,COMMON,Y
1,postgresql,postgres,192.168.110.111,5432,TAEYEON,TAEYEON,../dataset2,COMMON,Y
2,mysql+pymysql,mariadb,192.168.110.111,3306,TAEYEON,TAEYEON,../dataset2,COMMON,Y


In [5]:
#list 형태가 아니기 때문에 접근하려면 loc 사용해야함.
dbList = indata.loc[(indata['USE_YN'] == 'Y')] 

for i in range(len(dbList)):
    indataDF = dbList.iloc[i]
    PREFIX = indataDF["PREFIX"]
    ID = indataDF["ID"]
    PW = indataDF["PW"]
    IP = indataDF["IP"]         
    PORT = indataDF["PORT"]
    DBNAME = indataDF["DBNAME"]
    
    if indataDF["PREFIX"] == "oracle+cx_oracle":
        #print("oracle")
        engine = create_engine(f"{PREFIX}://{ID}:{PW}@{IP}:{PORT}/{DBNAME}")
    elif indataDF["PREFIX"] == "postgresql":
        #print("postgres")
        engine = create_engine(f"{PREFIX}://{ID}:{PW}@{IP}:{PORT}/{DBNAME}")
    elif indataDF["PREFIX"] == "mysql+pymysql":
        #print("maria") 
        engine = create_engine(f"{PREFIX}://{ID}:{PW}@{IP}:{PORT}/{DBNAME}")
    else :
        print("해당되는 값 없음.")

In [6]:
# if indata.loc[(indata['USE_YN'] == 'Y') & (indata['PREFIX'] == 'oracle+cx_oracle')]:
#     for i in range(len(indata.columns)):
#         eachDB = dbList.loc[0]

#         PREFIX = eachDB.dbPrefix
#         ID = eachDB.dbId
#         PW = eachDB.dbPw
#         IP = eachDB.dbIp         
#         PORT = eachDB.dbPort
#         DBNAME = eachDB.dbName
#         engine = create_engine(f"{PREFIX}://{ID}:{PW}@{IP}:{PORT}/{DBNAME}")

In [7]:
# if indata.loc[indata['prefix'] == 'oracle+cx_oracle'].shape[0] > 0:
#     for i in range(len(indata.columns)):
#         print(indata.iloc[0, i])
# elif indata.loc[indata['prefix'] == 'postgresql'].shape[0] > 0:
#     for i in range(len(indata.columns)):
#         print(indata.iloc[1, i])
# elif indata.loc[indata['prefix'] == 'mysql+pymysql'].shape[0] > 0:
#     for i in range(len(indata.columns)):
#         print(indata.iloc[2, i])

In [8]:
filepath = "../dataset2/"
lists = os.listdir(filepath)
lists

['.ipynb_checkpoints',
 'CHEON.csv',
 'CHESUNGBUN.csv',
 'CHOLESTEROL.csv',
 'electricityConsumptionAndProductioction.csv',
 'etl_param.csv',
 'GOMILDO_CHOLESTEROL.csv',
 'HYULAP.csv',
 'HYULDANG.csv',
 'JEOMILDO_CHOLESTEROL.csv',
 'JUNGSUNGJIBANG.csv',
 'PEEUM.csv',
 'SANSOPOHWADO.csv',
 'SIMBAKSU.csv',
 'SIMJANGEUM.csv',
 'US_honey_production_dataset.csv']

In [9]:
def csvToTable():
    try:
        for eachFile in lists:
            try:
                inData = pd.read_csv(filepath + eachFile)  # csv 파일 읽기
            except UnicodeDecodeError:
                # UnicodeDecodeError가 발생하면 ms949로 인코딩하여 다시 시도
                inData = pd.read_csv(filepath + eachFile, encoding='ms949')

            objColumns = list(inData.columns)
            tableName = eachFile[:-4]  # eachFile에서 .csv이름 제외하고 저장
            tableName = tableName.lower()
            
            # db로 저장하기
            typeDict={}
            
            for column in objColumns:
                if inData[column].dtypes  == 'float':
                    typeDict[column] = types.Float
                    #inData[column] = inData[column].fillna(0).astype(int)
                    #typeDict[column] = types.VARCHAR(100)
                elif inData[column].dtypes  == 'object':
                    typeDict[column] = types.VARCHAR(100)
            inData.to_sql(name=tableName, if_exists="replace", con=engine,dtype=typeDict, index=False)    
            
    except Exception as e:
        print("에러발생" + tableName)
        print(inData)
        print(typeDict)

csvToTable()

UnboundLocalError: local variable 'tableName' referenced before assignment